## Inverted Index with Cosine Similarity and TFIDF

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-317e  GCE       3                                             RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import builtins
from contextlib import closing

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*


-rw-r--r-- 1 root root 247882 Mar  2 22:44 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '208564682ri'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)

for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
body_data = parquetFile.select("text", "id").rdd


In [ ]:
# Count number of wiki pages
parquetFile.count()

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import InvertedIndex

# Building an inverted index

Remove stop words, tokenize and reduce words

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:'''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [token for token in tokens if token not in all_stopwords]
    word_counter = Counter(tokens)
    res_list = [(token, (id, tf)) for token, tf in word_counter.items()]
    return res_list

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.'''
    return sorted(unsorted_pl)

Calculate df and write to the bucket

In [ ]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.'''
  return postings.map(lambda x: (x[0], len(x[1]))).reduceByKey(lambda a, b: a + b)


NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class. '''
    bucket_sorted = postings.groupBy(lambda x: token2bucket_id(x[0]))
    posting_locations = bucket_sorted.map(lambda x: (InvertedIndex.write_a_posting_list(x ,".", bucket_name)))
    return posting_locations

In [ ]:
def tfIdf(term_freq, df, doc_len, N):
    tf = term_freq / doc_len
    idf = math.log(N / df)
    return tf * idf

body/title index

In [ ]:
body=body_data
index_name = "body_index"

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
#create the possting lists
word_counts = body.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>50)

#calculate df
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# calculate tf
tf = postings_filtered.flatMap(lambda x: [(post[0], (x[0],post[1])) for post in x[1]] )
tf = tf.groupByKey().mapValues(list)
documentes_length_RDD = tf.map(lambda x: (x[0],builtins.sum([tp[1] for tp in x[1]])))
documentes_length = documentes_length_RDD.collectAsMap()
# calculate tf-idf
tf_idf = tf.flatMap(lambda x: [(x[0], (curr[0], tfIdf(curr[1], inverted.df[curr[0]], documentes_length[x[0]], inverted.N))) for curr in x[1]]).groupByKey()
tf_idf = tf_idf.mapValues(lambda x: {tp[0]: tp[1] for tp in x})
# calculate cosine similarity
cosim = tf_idf.map(lambda x: (x[0],(doc_lens[x[0]],builtins.sum(math.pow(y,2) for y in x[1].values() ))))

# partition posting lists and write out
_ = partition_postings_and_write(postings).collect()
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add df and the cosine similarity dictionary to the inverted index
inverted.df = w2df_dict
inverted.similarity = cosim.collectAsMap()



Write as pickle to the bucket

In [ ]:
# write the global stats out
inverted.write_index('.', index_name, bucket_name)
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/body/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst